# Knapsack Heuristics Calculator and Analyzer

Complete system for evaluating greedy heuristics on 0/1 Knapsack Problem instances.

**Features:**
- 8 greedy heuristics with tie-breaking mechanisms
- Dynamic programming optimal solver
- Parallel processing support
- Comprehensive statistical analysis
- Automated report generation
- Pydantic validation for data integrity
- Unit testing suite

## 1. Imports and Dependencies

In [23]:
import os
import sys
import time
from datetime import datetime
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Protocol
from abc import ABC, abstractmethod
import logging
from concurrent.futures import ProcessPoolExecutor, as_completed
from functools import partial

import pandas as pd
import numpy as np
import pytz
import psutil

from numba import jit
from pydantic import BaseModel, Field, field_validator, ConfigDict, model_validator
from pydantic_settings import BaseSettings

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)
logger = logging.getLogger(__name__)

try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

try:
    import cpuinfo
    HAS_CPUINFO = True
except ImportError:
    HAS_CPUINFO = False
    logger.debug("cpuinfo not available, will use psutil only for hardware info")

## 2. Configuration Management

In [24]:
class ExperimentConfig(BaseSettings):
    model_config = ConfigDict(env_prefix='KP_')

    capacity: int = Field(
        default=64,
        gt=0,
        description="Knapsack capacity for all instances"
    )

    data_dir: str = Field(
        default='/content/drive/MyDrive/PlataZarateBayliss-Instances_extracted_csv',
        description="Directory containing instance CSV files"
    )

    output_dir: str = Field(
        default='/content',
        description="Directory for output files"
    )

    pattern: str = Field(
        default='*.csv',
        description="File pattern for instance files"
    )

    compute_optimal: bool = Field(
        default=True,
        description="Whether to compute optimal solutions via DP"
    )

    timezone: str = Field(
        default='America/Monterrey',
        description="Timezone for timestamps"
    )

    log_level: str = Field(
        default='INFO',
        description="Logging level (DEBUG, INFO, WARNING, ERROR)"
    )

    random_seed: Optional[int] = Field(
        default=None,
        description="Random seed for reproducibility (future use)"
    )

    max_workers: Optional[int] = Field(
        default=None,
        description="Max workers for parallel processing (None = auto-detect from CPU count)"
    )

    enable_parallel: bool = Field(
        default=True,
        description="Enable parallel processing for multiple instances"
    )

    @field_validator('max_workers')
    @classmethod
    def validate_max_workers(cls, v: Optional[int]) -> Optional[int]:
        if v is not None:
            cpu_count = psutil.cpu_count(logical=True)
            if v > cpu_count * 2:
                logger.warning(
                    f"max_workers={v} exceeds 2x CPU count ({cpu_count}). "
                    f"This may cause performance degradation."
                )
            if v < 1:
                raise ValueError("max_workers must be >= 1")
        return v

    @field_validator('data_dir', 'output_dir')
    @classmethod
    def validate_directory(cls, v: str) -> str:
        path = Path(v)
        if not path.exists():
            logger.warning(f"Directory {v} does not exist, will attempt to create")
        return v

    @field_validator('log_level')
    @classmethod
    def validate_log_level(cls, v: str) -> str:
        valid_levels = {'DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'}
        v_upper = v.upper()
        if v_upper not in valid_levels:
            raise ValueError(f"Log level must be one of {valid_levels}")
        return v_upper

    def setup_logging(self):
        logging.getLogger().setLevel(self.log_level)
        logger.setLevel(self.log_level)

    def to_dict(self) -> dict:
        return {
            'capacity': self.capacity,
            'data_dir': self.data_dir,
            'output_dir': self.output_dir,
            'pattern': self.pattern,
            'compute_optimal': self.compute_optimal,
            'timezone': self.timezone,
            'log_level': self.log_level,
            'random_seed': self.random_seed,
            'max_workers': self.max_workers,
            'python_version': f"{sys.version_info.major}.{sys.version_info.minor}.{sys.version_info.micro}",
            'numpy_version': np.__version__,
            'pandas_version': pd.__version__,
            'timestamp': datetime.now(pytz.timezone(self.timezone)).isoformat()
        }

    def save_to_file(self, filepath: str):
        import json
        config_dict = self.to_dict()
        with open(filepath, 'w') as f:
            json.dump(config_dict, f, indent=2)
        logger.info(f"Configuration saved to {filepath}")

    @classmethod
    def load_from_file(cls, filepath: str) -> 'ExperimentConfig':
        import json
        with open(filepath, 'r') as f:
            config_dict = json.load(f)
        return cls(**{k: v for k, v in config_dict.items() if k in cls.model_fields})

## 3. Data Models with Pydantic Validation

In [25]:
class KnapsackInstance(BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)

    profits: np.ndarray
    weights: np.ndarray
    capacity: int = Field(gt=0, description="Knapsack capacity must be positive")
    name: str = Field(min_length=1, description="Instance name")

    @field_validator('profits', 'weights')
    @classmethod
    def validate_arrays(cls, v: np.ndarray) -> np.ndarray:
        if not isinstance(v, np.ndarray):
            raise ValueError("Must be numpy array")
        if v.dtype != np.int64:
            v = v.astype(np.int64)
        if len(v) == 0:
            raise ValueError("Array cannot be empty")
        if np.any(v <= 0):
            raise ValueError(f"All values must be positive, found {np.sum(v <= 0)} non-positive values")
        return v

    @model_validator(mode='after')
    def validate_arrays_match(self):
        if len(self.profits) != len(self.weights):
            raise ValueError(
                f"Mismatched array lengths: profits={len(self.profits)}, weights={len(self.weights)}"
            )

        items_that_fit = np.sum(self.weights <= self.capacity)
        if items_that_fit == 0:
            raise ValueError(
                f"No items can fit in capacity {self.capacity}: "
                f"minimum weight is {np.min(self.weights)}, all {len(self.weights)} items exceed capacity"
            )

        max_sum = np.sum(self.profits)
        if max_sum > np.iinfo(np.int64).max // 2:
            raise ValueError(
                f"Risk of overflow: sum of profits {max_sum} exceeds safe threshold "
                f"{np.iinfo(np.int64).max // 2}"
            )
        return self

    @property
    def num_items(self) -> int:
        return len(self.profits)

    @classmethod
    def from_csv(cls, filepath: str, capacity: int) -> 'KnapsackInstance':
        if not Path(filepath).exists():
            raise FileNotFoundError(f"File not found: {filepath}")

        df = pd.read_csv(filepath)

        required_columns = {'profit', 'weight'}
        if not required_columns.issubset(df.columns):
            raise ValueError(f"CSV must contain columns: {required_columns}, found: {df.columns.tolist()}")

        return cls(
            profits=df['profit'].values.astype(np.int64),
            weights=df['weight'].values.astype(np.int64),
            capacity=capacity,
            name=Path(filepath).stem
        )


class HeuristicResult(BaseModel):
    heuristic_name: str = Field(min_length=1)
    solution_value: int = Field(ge=0, description="Solution value must be non-negative")
    execution_time: float = Field(ge=0, description="Execution time must be non-negative")

    @field_validator('execution_time')
    @classmethod
    def validate_execution_time(cls, v: float) -> float:
        if v > 3600:
            logger.warning(f"Execution time {v}s exceeds 1 hour, possible timing error")
        return v


class InstanceResult(BaseModel):
    instance_name: str = Field(min_length=1)
    capacity: int = Field(gt=0)
    heuristic_results: Dict[str, HeuristicResult]
    optimal_value: Optional[int] = Field(default=None, ge=0)
    optimal_time: Optional[float] = Field(default=None, ge=0)

    @model_validator(mode='after')
    def validate_results(self):
        if self.optimal_value is not None:
            for name, result in self.heuristic_results.items():
                if result.solution_value > self.optimal_value:
                    raise ValueError(
                        f"CRITICAL BUG DETECTED: Heuristic {name} returned {result.solution_value} > "
                        f"optimal {self.optimal_value}. This indicates a bug in the implementation."
                    )
        return self

    def to_dict(self) -> dict:
        result = {
            'instance': self.instance_name,
            'capacity': self.capacity
        }
        for name, heur_result in self.heuristic_results.items():
            result[name] = heur_result.solution_value
        if self.optimal_value is not None:
            result['optimal'] = self.optimal_value
        return result

    def to_json(self) -> str:
        return self.model_dump_json(indent=2)

    @classmethod
    def from_json(cls, json_str: str) -> 'InstanceResult':
        return cls.model_validate_json(json_str)

## 4. Numba-Optimized Core Functions

In [26]:
@jit(nopython=True)
def _greedy_selection(profits: np.ndarray, weights: np.ndarray,
                      capacity: np.int64, indices: np.ndarray) -> np.int64:
    """
    Greedy knapsack selection with pre-determined item order.
    """
    total_profit = 0
    total_weight = 0
    for i in indices:
        if total_weight + weights[i] <= capacity:
            total_weight += weights[i]
            total_profit += profits[i]
    return total_profit


@jit(nopython=True)
def _compute_ratios(profits: np.ndarray, weights: np.ndarray) -> np.ndarray:
    n = len(profits)
    ratios = np.empty(n, dtype=np.float64)
    for i in range(n):
        if weights[i] > 0:
            ratios[i] = profits[i] / weights[i]
        else:
            ratios[i] = 1e308 if profits[i] > 0 else 0.0
    return ratios


@jit(nopython=True)
def _knapsack_dp(profits: np.ndarray, weights: np.ndarray, capacity: np.int64) -> np.int64:
    n: np.int64 = len(profits)
    prev = np.zeros(capacity + 1, dtype=np.int64)
    curr = np.zeros(capacity + 1, dtype=np.int64)

    for i in range(n):
        for w in range(capacity + 1):
            if weights[i] <= w:
                curr[w] = max(prev[w], prev[w - weights[i]] + profits[i])
            else:
                curr[w] = prev[w]
        prev, curr = curr, prev

    return prev[capacity]

## 5. Heuristic Implementations (Strategy Pattern)

In [27]:
class Heuristic(ABC):
    @abstractmethod
    def solve(self, instance: KnapsackInstance) -> int:
        pass

    @property
    @abstractmethod
    def name(self) -> str:
        pass


class DefaultHeuristic(Heuristic):
    @property
    def name(self) -> str:
        return 'default'

    def solve(self, instance: KnapsackInstance) -> int:
        indices = np.arange(len(instance.profits), dtype=np.int64)
        return _greedy_selection(instance.profits, instance.weights, instance.capacity, indices)


class MaxProfitHeuristic(Heuristic):
    @property
    def name(self) -> str:
        return 'max_profit'

    def solve(self, instance: KnapsackInstance) -> int:
        indices = np.argsort(instance.profits)[::-1].astype(np.int64)
        return _greedy_selection(instance.profits, instance.weights, instance.capacity, indices)


class MaxProfitPerWeightHeuristic(Heuristic):
    @property
    def name(self) -> str:
        return 'max_profit_per_weight'

    def solve(self, instance: KnapsackInstance) -> int:
        ratios = _compute_ratios(instance.profits, instance.weights)
        indices = np.argsort(ratios)[::-1].astype(np.int64)
        return _greedy_selection(instance.profits, instance.weights, instance.capacity, indices)


class MinWeightHeuristic(Heuristic):
    @property
    def name(self) -> str:
        return 'min_weight'

    def solve(self, instance: KnapsackInstance) -> int:
        indices = np.argsort(instance.weights).astype(np.int64)
        return _greedy_selection(instance.profits, instance.weights, instance.capacity, indices)


class MaxProfitPerWeightTiebreakProfitHeuristic(Heuristic):
    @property
    def name(self) -> str:
        return 'max_profit_per_weight_tiebreak_profit'

    def solve(self, instance: KnapsackInstance) -> int:
        ratios = _compute_ratios(instance.profits, instance.weights)
        indices = np.lexsort((instance.profits, ratios))[::-1].astype(np.int64)
        return _greedy_selection(instance.profits, instance.weights, instance.capacity, indices)


class MaxProfitPerWeightTiebreakWeightHeuristic(Heuristic):
    @property
    def name(self) -> str:
        return 'max_profit_per_weight_tiebreak_weight'

    def solve(self, instance: KnapsackInstance) -> int:
        ratios = _compute_ratios(instance.profits, instance.weights)
        indices = np.lexsort((instance.weights, -ratios)).astype(np.int64)
        return _greedy_selection(instance.profits, instance.weights, instance.capacity, indices)


class MaxProfitTiebreakWeightHeuristic(Heuristic):
    @property
    def name(self) -> str:
        return 'max_profit_tiebreak_weight'

    def solve(self, instance: KnapsackInstance) -> int:
        indices = np.lexsort((instance.weights, -instance.profits)).astype(np.int64)
        return _greedy_selection(instance.profits, instance.weights, instance.capacity, indices)


class MinWeightTiebreakProfitHeuristic(Heuristic):
    @property
    def name(self) -> str:
        return 'min_weight_tiebreak_profit'

    def solve(self, instance: KnapsackInstance) -> int:
        indices = np.lexsort((-instance.profits, instance.weights)).astype(np.int64)
        return _greedy_selection(instance.profits, instance.weights, instance.capacity, indices)

## 6. Optimal Solver and Heuristic Registry

In [28]:
class OptimalSolver:
    def solve(self, instance: KnapsackInstance) -> int:
        return _knapsack_dp(instance.profits, instance.weights, instance.capacity)

    def solve_with_time(self, instance: KnapsackInstance) -> Tuple[int, float]:
        start_time = time.perf_counter()
        result = _knapsack_dp(instance.profits, instance.weights, instance.capacity)
        execution_time = time.perf_counter() - start_time
        return result, execution_time


class HeuristicRegistry:
    def __init__(self):
        self._heuristics: Dict[str, Heuristic] = {}
        self._register_defaults()

    def _register_defaults(self):
        default_heuristics = [
            DefaultHeuristic(),
            MaxProfitHeuristic(),
            MaxProfitPerWeightHeuristic(),
            MinWeightHeuristic(),
            MaxProfitPerWeightTiebreakProfitHeuristic(),
            MaxProfitPerWeightTiebreakWeightHeuristic(),
            MaxProfitTiebreakWeightHeuristic(),
            MinWeightTiebreakProfitHeuristic()
        ]
        for heuristic in default_heuristics:
            self.register(heuristic)

    def register(self, heuristic: Heuristic):
        self._heuristics[heuristic.name] = heuristic

    def get(self, name: str) -> Heuristic:
        return self._heuristics[name]

    def get_all(self) -> Dict[str, Heuristic]:
        return self._heuristics.copy()

    def get_names(self) -> List[str]:
        return list(self._heuristics.keys())

## 7. Experiment Executor with Parallel Processing

In [29]:
def _process_instance_wrapper(filepath: str, capacity: int, compute_optimal: bool,
                              registry: HeuristicRegistry, solver: OptimalSolver) -> Optional[InstanceResult]:
    try:
        instance = KnapsackInstance.from_csv(filepath, capacity)

        heuristic_results = {}
        for name, heuristic in registry.get_all().items():
            start_time = time.perf_counter()
            solution_value = heuristic.solve(instance)
            execution_time = time.perf_counter() - start_time

            heuristic_results[name] = HeuristicResult(
                heuristic_name=name,
                solution_value=solution_value,
                execution_time=execution_time
            )

        optimal_value = None
        optimal_time = None
        if compute_optimal:
            optimal_value, optimal_time = solver.solve_with_time(instance)

        return InstanceResult(
            instance_name=instance.name,
            capacity=instance.capacity,
            heuristic_results=heuristic_results,
            optimal_value=optimal_value,
            optimal_time=optimal_time
        )
    except Exception as e:
        logger.error(f"Failed to process {filepath}: {e}")
        return None


class ExperimentExecutor:
    def __init__(self, registry: HeuristicRegistry, optimal_solver: OptimalSolver):
        self.registry = registry
        self.optimal_solver = optimal_solver
        self.logger = logging.getLogger(__name__ + '.ExperimentExecutor')

    def execute_single_instance(self, instance: KnapsackInstance,
                                compute_optimal: bool = False) -> InstanceResult:
        heuristic_results = {}

        for name, heuristic in self.registry.get_all().items():
            start_time = time.perf_counter()
            solution_value = heuristic.solve(instance)
            execution_time = time.perf_counter() - start_time

            heuristic_results[name] = HeuristicResult(
                heuristic_name=name,
                solution_value=solution_value,
                execution_time=execution_time
            )

        optimal_value = None
        optimal_time = None
        if compute_optimal:
            optimal_value, optimal_time = self.optimal_solver.solve_with_time(instance)

        return InstanceResult(
            instance_name=instance.name,
            capacity=instance.capacity,
            heuristic_results=heuristic_results,
            optimal_value=optimal_value,
            optimal_time=optimal_time
        )

    def execute_directory(self, directory: str, capacity: int,
                         pattern: str = '*.csv',
                         compute_optimal: bool = False,
                         parallel: bool = False,
                         max_workers: Optional[int] = None) -> List[InstanceResult]:
        directory_path = Path(directory)
        instance_files = sorted([
            f for f in directory_path.glob(pattern)
            if f.name.startswith(('TRAIN', 'TEST'))
        ])

        total = len(instance_files)
        self.logger.info(f"Found {total} instances in {directory}")

        if parallel:
            return self._execute_parallel(instance_files, capacity, compute_optimal, max_workers)
        else:
            return self._execute_sequential(instance_files, capacity, compute_optimal)

    def _execute_sequential(self, instance_files: List[Path], capacity: int,
                           compute_optimal: bool) -> List[InstanceResult]:
        results = []
        total = len(instance_files)

        for idx, filepath in enumerate(instance_files, 1):
            try:
                instance = KnapsackInstance.from_csv(str(filepath), capacity)
                result = self.execute_single_instance(instance, compute_optimal)
                results.append(result)

                if idx % 10 == 0:
                    self.logger.info(f"Processed {idx}/{total}")
                    print(f"Processed {idx}/{total}")

            except Exception as e:
                self.logger.error(f"Failed to process {filepath}: {e}")

        self.logger.info(f"Completed: {len(results)}/{total} instances")
        return results

    def _execute_parallel(self, instance_files: List[Path], capacity: int,
                         compute_optimal: bool, max_workers: Optional[int] = None) -> List[InstanceResult]:
        total = len(instance_files)

        if max_workers is None:
            max_workers = min(32, (psutil.cpu_count(logical=True) or 1) + 4)

        self.logger.info(f"Starting parallel execution with {max_workers} workers")
        print(f"Processing {total} instances in parallel ({max_workers} workers)...")

        results = []
        completed = 0

        process_func = partial(
            _process_instance_wrapper,
            capacity=capacity,
            compute_optimal=compute_optimal,
            registry=self.registry,
            solver=self.optimal_solver
        )

        with ProcessPoolExecutor(max_workers=max_workers) as executor:
            future_to_filepath = {
                executor.submit(process_func, str(filepath)): filepath
                for filepath in instance_files
            }

            for future in as_completed(future_to_filepath):
                filepath = future_to_filepath[future]
                try:
                    result = future.result()
                    if result is not None:
                        results.append(result)
                    completed += 1

                    if completed % 10 == 0:
                        self.logger.info(f"Completed {completed}/{total}")
                        print(f"Completed {completed}/{total}")

                except Exception as e:
                    self.logger.error(f"Exception processing {filepath}: {e}")
                    completed += 1

        results.sort(key=lambda r: r.instance_name)

        self.logger.info(f"Parallel execution completed: {len(results)}/{total} instances")
        return results

## 8. Results Converter and Statistical Analyzer

In [30]:
class ResultsConverter:
    @staticmethod
    def to_dataframe(results: List[InstanceResult]) -> pd.DataFrame:
        records = [result.to_dict() for result in results]
        return pd.DataFrame(records)

    @staticmethod
    def extract_execution_times(results: List[InstanceResult]) -> Dict[str, List[float]]:
        execution_times = {}

        for result in results:
            for name, heur_result in result.heuristic_results.items():
                if name not in execution_times:
                    execution_times[name] = []
                execution_times[name].append(heur_result.execution_time)

            if result.optimal_time is not None:
                if 'optimal' not in execution_times:
                    execution_times['optimal'] = []
                execution_times['optimal'].append(result.optimal_time)

        return execution_times


class StatisticalAnalyzer:
    """
    - BASIC_HEURISTICS: Contains the 4 basic heuristics without tiebreak mechanisms
    - TIEBREAK_HEURISTICS: Despite the name, this includes SOME heuristics WITHOUT
      tiebreak ('default', 'max_profit_per_weight') alongside actual tiebreak heuristics.

    The "TIEBREAK_HEURISTICS" is the original design for specific comparison purposes with previous works.
    """
    BASIC_HEURISTICS = ['default', 'max_profit', 'max_profit_per_weight', 'min_weight']
    TIEBREAK_HEURISTICS = ['default', 'max_profit_tiebreak_weight',
                          'min_weight_tiebreak_profit', 'max_profit_per_weight']

    @staticmethod
    def compute_statistics(df: pd.DataFrame, heuristic_columns: List[str]) -> pd.DataFrame:
        stats = []
        for col in heuristic_columns:
            values = df[col].dropna()
            stats.append({
                'heuristic': col,
                'mean': values.mean(),
                'std': values.std(),
                'min': values.min(),
                'max': values.max(),
                'median': values.median()
            })
        return pd.DataFrame(stats)

    @staticmethod
    def compute_gap_statistics(df: pd.DataFrame, heuristic_columns: List[str],
                              optimal_column: str = 'optimal') -> pd.DataFrame:
        if optimal_column not in df.columns:
            return pd.DataFrame()

        gap_stats = []
        for col in heuristic_columns:
            gaps = ((df[optimal_column] - df[col]) / df[optimal_column] * 100)
            gap_stats.append({
                'heuristic': col,
                'mean_gap': gaps.mean(),
                'std_gap': gaps.std(),
                'min_gap': gaps.min(),
                'max_gap': gaps.max(),
                'median_gap': gaps.median(),
                'optimal_found': (gaps == 0).sum()
            })
        return pd.DataFrame(gap_stats)

    @staticmethod
    def identify_dominant_heuristic(df: pd.DataFrame,
                                   heuristic_columns: List[str]) -> pd.Series:
        best_heuristics = df[heuristic_columns].idxmax(axis=1)
        return best_heuristics.value_counts()

    @staticmethod
    def identify_dominant_basic_heuristics(df: pd.DataFrame) -> pd.Series:
        available_basic = [col for col in StatisticalAnalyzer.BASIC_HEURISTICS
                          if col in df.columns]
        if not available_basic:
            return pd.Series(dtype='int64')
        best_heuristics = df[available_basic].idxmax(axis=1)
        return best_heuristics.value_counts()

    @staticmethod
    def identify_dominant_tiebreak_heuristics(df: pd.DataFrame) -> pd.Series:
        available_tiebreak = [col for col in StatisticalAnalyzer.TIEBREAK_HEURISTICS
                             if col in df.columns]
        if not available_tiebreak:
            return pd.Series(dtype='int64')
        best_heuristics = df[available_tiebreak].idxmax(axis=1)
        return best_heuristics.value_counts()

    @staticmethod
    def compute_tiebreak_gap_analysis(df: pd.DataFrame) -> List[dict]:
        if 'optimal' not in df.columns:
            return []

        tiebreak_heuristics = StatisticalAnalyzer.TIEBREAK_HEURISTICS
        tiebreak_available = [col for col in tiebreak_heuristics if col in df.columns]

        if not tiebreak_available:
            return []

        tiebreak_instance_counts = {}
        for instance in df['instance']:
            best_tiebreak = df.loc[df['instance'] == instance, tiebreak_available].idxmax(axis=1).values[0]
            tiebreak_instance_counts.setdefault(best_tiebreak, []).append(instance)

        tiebreak_gap_stats_dominated = []
        for heuristic in tiebreak_available:
            if heuristic in tiebreak_instance_counts:
                instances = tiebreak_instance_counts[heuristic]
                if instances:
                    gaps = []
                    for instance in instances:
                        optimal_val = df.loc[df['instance'] == instance, 'optimal'].values[0]
                        heuristic_val = df.loc[df['instance'] == instance, heuristic].values[0]

                        if optimal_val > 0:
                            gap = ((optimal_val - heuristic_val) / optimal_val) * 100
                            gaps.append(gap)

                    if gaps:
                        tiebreak_gap_stats_dominated.append({
                            'heuristic': heuristic,
                            'instances': len(instances),
                            'mean_gap': np.mean(gaps),
                            'std_gap': np.std(gaps) if len(gaps) > 1 else 0,
                            'min_gap': np.min(gaps),
                            'max_gap': np.max(gaps),
                            'optimal_found': sum(1 for g in gaps if g == 0)
                        })

        return tiebreak_gap_stats_dominated

    @staticmethod
    def compute_comparison(df: pd.DataFrame, heuristic_columns: List[str]) -> pd.DataFrame:
        comparison = df[['instance'] + heuristic_columns].copy()
        comparison['best_heuristic'] = comparison[heuristic_columns].idxmax(axis=1)
        comparison['best_value'] = comparison[heuristic_columns].max(axis=1)
        comparison['worst_value'] = comparison[heuristic_columns].min(axis=1)
        comparison['value_range'] = comparison['best_value'] - comparison['worst_value']
        return comparison

## 9. Report Formatter and Generator

In [31]:
class ReportFormatter:
    def __init__(self):
        self.mty_tz = pytz.timezone('America/Monterrey')
        self.current_time = datetime.now(self.mty_tz)

    def format_header(self, title: str, has_optimal: bool = False) -> List[str]:
        full_title = title
        if has_optimal:
            full_title += " (WITH OPTIMAL)"

        return [
            "=" * 80,
            full_title,
            "=" * 80
        ]

    def format_section(self, title: str) -> List[str]:
        return ["", title.upper(), "-" * 80]

    def format_execution_info(self) -> List[str]:
        return [
            f"Date: {self.current_time.strftime('%Y-%m-%d')}",
            f"Time: {self.current_time.strftime('%H:%M:%S')}",
            f"Timezone: America/Monterrey (UTC{self.current_time.strftime('%z')})"
        ]

    def format_hardware_info(self) -> List[str]:
        lines = []

        try:
            cpu_count = psutil.cpu_count(logical=True)
            cpu_count_physical = psutil.cpu_count(logical=False)
            memory = psutil.virtual_memory()
            cpu_freq = psutil.cpu_freq()

            lines.extend([
                f"CPU Cores: {cpu_count_physical} physical, {cpu_count} logical",
                f"Total Memory: {memory.total / (1024**3):.1f} GB",
                f"Available Memory: {memory.available / (1024**3):.1f} GB",
                f"Memory Usage: {memory.percent}%"
            ])

            if cpu_freq:
                lines.append(f"CPU Frequency: {cpu_freq.current:.0f} MHz")

        except Exception as e:
            logger.warning(f"Could not retrieve basic hardware info: {e}")

        if HAS_CPUINFO:
            try:
                cpu_info = cpuinfo.get_cpu_info()
                brand = cpu_info.get('brand_raw', cpu_info.get('brand', 'Unknown'))
                lines.append(f"CPU Model: {brand}")
            except Exception as e:
                logger.debug(f"cpuinfo failed, skipping detailed CPU info: {e}")
        else:
            lines.append("CPU Model: Install 'py-cpuinfo' for detailed CPU information")

        return lines

    def format_statistics_table(self, stats_df: pd.DataFrame) -> List[str]:
        lines = []
        header = f"{'Heuristic':<40} {'Mean':>10} {'Std':>10} {'Min':>10} {'Max':>10}"
        lines.append(header)
        lines.append("-" * 80)

        for _, row in stats_df.iterrows():
            if row['heuristic'] != 'optimal':
                lines.append(
                    f"{row['heuristic']:<40} "
                    f"{row['mean']:>10.2f} "
                    f"{row['std']:>10.2f} "
                    f"{row['min']:>10.0f} "
                    f"{row['max']:>10.0f}"
                )
        return lines

    def format_gap_table(self, gap_stats_df: pd.DataFrame, total_instances: int) -> List[str]:
        lines = []
        header = (f"{'Heuristic':<40} {'Mean Gap %':>12} {'Std Gap %':>12} "
                 f"{'Min Gap %':>12} {'Max Gap %':>12} {'Optimal Found':>15}")
        lines.append(header)
        lines.append("-" * 130)

        sorted_stats = gap_stats_df.sort_values('mean_gap')
        for _, row in sorted_stats.iterrows():
            lines.append(
                f"{row['heuristic']:<40} "
                f"{row['mean_gap']:>12.2f} "
                f"{row['std_gap']:>12.2f} "
                f"{row['min_gap']:>12.2f} "
                f"{row['max_gap']:>12.2f} "
                f"{row['optimal_found']:>8}/{total_instances}"
            )

        best_heuristic = sorted_stats.iloc[0]['heuristic']
        best_gap = sorted_stats.iloc[0]['mean_gap']
        worst_heuristic = sorted_stats.iloc[-1]['heuristic']
        worst_gap = sorted_stats.iloc[-1]['mean_gap']

        lines.append("")
        lines.append(f"Best gap: {best_heuristic} ({best_gap:.2f}% avg)")
        lines.append(f"Worst gap: {worst_heuristic} ({worst_gap:.2f}% avg)")

        return lines

    def format_tiebreak_gap_table(self, tiebreak_stats: List[dict]) -> List[str]:
        if not tiebreak_stats:
            return ["No tie-break dominance data available"]

        lines = []
        header = (f"{'Heuristic':<40} {'Instances':>10} {'Mean Gap %':>12} {'Std Gap %':>12} "
                 f"{'Min Gap %':>12} {'Max Gap %':>12} {'Optimal Found':>15}")
        lines.append(header)
        lines.append("-" * 140)

        sorted_stats = sorted(tiebreak_stats, key=lambda x: x['mean_gap'])
        for stats in sorted_stats:
            lines.append(
                f"{stats['heuristic']:<40} "
                f"{stats['instances']:>10} "
                f"{stats['mean_gap']:>12.2f} "
                f"{stats['std_gap']:>12.2f} "
                f"{stats['min_gap']:>12.2f} "
                f"{stats['max_gap']:>12.2f} "
                f"{stats['optimal_found']:>8}/{stats['instances']}"
            )

        if sorted_stats:
            best = sorted_stats[0]
            worst = sorted_stats[-1]
            lines.append("")
            lines.append(f"Best tie-break gap (when dominated): {best['heuristic']} ({best['mean_gap']:.2f}% avg, {best['instances']} instances)")
            lines.append(f"Worst tie-break gap (when dominated): {worst['heuristic']} ({worst['mean_gap']:.2f}% avg, {worst['instances']} instances)")

        return lines

    def format_dominant_heuristics(self, dominant: pd.Series, total: int, category: str = "ALL") -> List[str]:
        if dominant.empty:
            return [f"No dominance data for {category} heuristics"]

        lines = [f"Number of times each {category} heuristic achieved best solution:", ""]
        for heuristic, count in dominant.sort_values(ascending=False).items():
            percentage = (count / total) * 100
            lines.append(f"  {heuristic:<40} {count:>6} instances ({percentage:>5.1f}%)")

        return lines

    def format_execution_times(self, avg_times: Dict[str, float],
                              total_times: Dict[str, float]) -> List[str]:
        lines = []
        header = f"{'Method':<40} {'Avg/Inst (μs)':>15} {'Total (ms)':>15}"
        lines.append(header)
        lines.append("-" * 80)

        for name in sorted(avg_times.keys()):
            if name != 'optimal':
                avg_us = avg_times[name] * 1_000_000
                total_ms = total_times.get(name, 0) * 1000
                lines.append(f"{name:<40} {avg_us:>15.3f} {total_ms:>15.3f}")

        if 'optimal' in avg_times:
            avg_us = avg_times['optimal'] * 1_000_000
            total_ms = total_times.get('optimal', 0) * 1000
            lines.append(f"{'DP Optimal Solver':<40} {avg_us:>15.3f} {total_ms:>15.3f}")

        return lines

In [32]:
class ReportGenerator:
    def __init__(self, formatter: ReportFormatter, analyzer: StatisticalAnalyzer):
        self.formatter = formatter
        self.analyzer = analyzer
        self.logger = logging.getLogger(__name__ + '.ReportGenerator')

    def generate(self, df: pd.DataFrame, execution_times: Dict[str, List[float]],
                config: dict, output_path: str) -> str:
        lines = []
        heuristic_columns = [col for col in df.columns
                           if col not in ['instance', 'capacity', 'optimal']]

        has_optimal = 'optimal' in df.columns

        lines.extend(self.formatter.format_header(
            "KNAPSACK HEURISTICS EXPERIMENT REPORT", has_optimal))

        lines.extend(self.formatter.format_section("EXECUTION INFORMATION"))
        lines.extend(self.formatter.format_execution_info())

        lines.extend(self.formatter.format_section("HARDWARE INFORMATION"))
        lines.extend(self.formatter.format_hardware_info())

        lines.extend(self.formatter.format_section("EXPERIMENT CONFIGURATION"))
        lines.extend(self._format_config(config, df))

        lines.extend(self.formatter.format_section("HEURISTICS EVALUATED"))
        lines.extend([f"{i}. {h}" for i, h in enumerate(heuristic_columns, 1)])

        stats_df = self.analyzer.compute_statistics(df, heuristic_columns)
        lines.extend(self.formatter.format_section("HEURISTIC PERFORMANCE STATISTICS"))
        lines.extend(self.formatter.format_statistics_table(stats_df))

        if has_optimal:
            gap_stats = self.analyzer.compute_gap_statistics(df, heuristic_columns)
            lines.extend(self.formatter.format_section("OPTIMALITY GAP ANALYSIS"))
            lines.extend(self.formatter.format_gap_table(gap_stats, len(df)))

        avg_times = {name: np.mean(times) for name, times in execution_times.items()}
        total_times = {name: np.sum(times) for name, times in execution_times.items()}
        lines.extend(self.formatter.format_section("EXECUTION TIME ANALYSIS"))
        lines.extend(self.formatter.format_execution_times(avg_times, total_times))

        dominant = self.analyzer.identify_dominant_heuristic(df, heuristic_columns)
        lines.extend(self.formatter.format_section("DOMINANT HEURISTIC ANALYSIS (ALL HEURISTICS)"))
        lines.extend(self.formatter.format_dominant_heuristics(dominant, len(df), "ALL"))

        dominant_basic = self.analyzer.identify_dominant_basic_heuristics(df)
        if not dominant_basic.empty:
            lines.extend(self.formatter.format_section("BASIC HEURISTICS DOMINANCE ANALYSIS"))
            lines.extend(self.formatter.format_dominant_heuristics(dominant_basic, len(df), "BASIC"))

        dominant_tiebreak = self.analyzer.identify_dominant_tiebreak_heuristics(df)
        if not dominant_tiebreak.empty:
            lines.extend(self.formatter.format_section("TIE-BREAK HEURISTICS DOMINANCE ANALYSIS"))
            lines.extend(self.formatter.format_dominant_heuristics(dominant_tiebreak, len(df), "TIE-BREAK"))

        if has_optimal:
            tiebreak_gap_stats = self.analyzer.compute_tiebreak_gap_analysis(df)
            if tiebreak_gap_stats:
                lines.extend(self.formatter.format_section("TIE-BREAK OPTIMALITY GAP ANALYSIS (Instances where each dominated)"))
                lines.extend(self.formatter.format_tiebreak_gap_table(tiebreak_gap_stats))

        lines.extend(["", "END OF REPORT", "=" * 80])

        report_text = '\n'.join(lines)
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(report_text)

        self.logger.info(f"Report generated: {output_path}")
        return report_text

    def _format_config(self, config: dict, df: pd.DataFrame) -> List[str]:
        return [
            f"Capacity: {config.get('capacity', 'N/A')}",
            f"Data directory: {config.get('data_dir', 'N/A')}",
            f"Total instances: {len(df)}",
            f"Compute optimal: {'optimal' in df.columns}",
            f"Parallel processing: {config.get('enable_parallel', False)}",
            f"Python version: {config.get('python_version', 'N/A')}",
            f"NumPy version: {np.__version__}",
            f"Pandas version: {pd.__version__}"
        ]

## 10. Utility Functions (I/O)

In [33]:
def save_results(df: pd.DataFrame, output_path: str, include_timestamp: bool = True,
                base_name: Optional[str] = None) -> str:
    if include_timestamp:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        path_obj = Path(output_path)

        if base_name:
            filename = f"{base_name}_{timestamp}.csv"
        else:
            filename = f"results_{timestamp}.csv"

        if path_obj.is_dir():
            final_path = path_obj / filename
        else:
            final_path = path_obj.parent / f"{path_obj.stem}_{timestamp}{path_obj.suffix}"
    else:
        final_path = Path(output_path)

    df.to_csv(final_path, index=False)
    logger.info(f"Saved results: {final_path}")
    return str(final_path)

## 11. Unit Tests

In [34]:
import unittest
import tempfile

class TestKnapsackInstance(unittest.TestCase):
    def test_valid_instance_creation(self):
        instance = KnapsackInstance(
            profits=np.array([10, 20, 30], dtype=np.int64),
            weights=np.array([5, 10, 15], dtype=np.int64),
            capacity=20,
            name='test'
        )
        self.assertEqual(instance.num_items, 3)
        self.assertEqual(instance.capacity, 20)

    def test_negative_capacity_raises_error(self):
        from pydantic import ValidationError
        with self.assertRaises(ValidationError):
            KnapsackInstance(
                profits=np.array([10, 20], dtype=np.int64),
                weights=np.array([5, 10], dtype=np.int64),
                capacity=-5,
                name='invalid'
            )


class TestOptimalSolver(unittest.TestCase):
    def setUp(self):
        self.solver = OptimalSolver()

    def test_known_optimal_solution(self):
        instance = KnapsackInstance(
            profits=np.array([60, 100, 120], dtype=np.int64),
            weights=np.array([10, 20, 30], dtype=np.int64),
            capacity=50,
            name='test'
        )
        result = self.solver.solve(instance)
        self.assertEqual(result, 220)


class TestHeuristics(unittest.TestCase):
    def setUp(self):
        self.instance = KnapsackInstance(
            profits=np.array([60, 100, 120], dtype=np.int64),
            weights=np.array([10, 20, 30], dtype=np.int64),
            capacity=50,
            name='test'
        )
        self.optimal_value = 220

    def test_all_heuristics_deterministic(self):
        heuristics = [
            MaxProfitHeuristic(),
            MaxProfitPerWeightHeuristic(),
            MinWeightHeuristic()
        ]

        for heuristic in heuristics:
            result1 = heuristic.solve(self.instance)
            result2 = heuristic.solve(self.instance)
            self.assertEqual(result1, result2,
                           f"{heuristic.name} is not deterministic")

    def test_heuristics_do_not_exceed_optimal(self):
        heuristics = [
            MaxProfitHeuristic(),
            MaxProfitPerWeightHeuristic(),
            MinWeightHeuristic()
        ]

        for heuristic in heuristics:
            result = heuristic.solve(self.instance)
            self.assertLessEqual(result, self.optimal_value,
                f"{heuristic.name} exceeded optimal value")

print("\nRun tests with: unittest.main(argv=[''], exit=False, verbosity=2)")


Run tests with: unittest.main(argv=[''], exit=False, verbosity=2)


### Run Unit Tests

In [35]:
# Run all unit tests
unittest.main(argv=[''], exit=False, verbosity=2)

test_all_heuristics_deterministic (__main__.TestHeuristics.test_all_heuristics_deterministic) ... ok
test_heuristics_do_not_exceed_optimal (__main__.TestHeuristics.test_heuristics_do_not_exceed_optimal) ... ok
test_negative_capacity_raises_error (__main__.TestKnapsackInstance.test_negative_capacity_raises_error) ... ok
test_valid_instance_creation (__main__.TestKnapsackInstance.test_valid_instance_creation) ... ok
test_known_optimal_solution (__main__.TestOptimalSolver.test_known_optimal_solution) ... ok

----------------------------------------------------------------------
Ran 5 tests in 0.485s

OK


## 12. Main Execution

### 12.1 Initialize Configuration

In [36]:
logger.info("Starting Knapsack Heuristics Experiment")

config = ExperimentConfig()
config.setup_logging()

logger.info(f"Configuration: {config.to_dict()}")
print("\n" + "="*80)
print("EXPERIMENT CONFIGURATION")
print("="*80)
for key, value in config.to_dict().items():
    print(f"{key:<25}: {value}")
print("="*80)

INFO:__main__:Starting Knapsack Heuristics Experiment
INFO:__main__:Configuration: {'capacity': 64, 'data_dir': '/content/drive/MyDrive/PlataZarateBayliss-Instances_extracted_csv', 'output_dir': '/content', 'pattern': '*.csv', 'compute_optimal': True, 'timezone': 'America/Monterrey', 'log_level': 'INFO', 'random_seed': None, 'max_workers': None, 'python_version': '3.12.12', 'numpy_version': '2.0.2', 'pandas_version': '2.2.2', 'timestamp': '2025-12-22T17:28:58.914188-06:00'}



EXPERIMENT CONFIGURATION
capacity                 : 64
data_dir                 : /content/drive/MyDrive/PlataZarateBayliss-Instances_extracted_csv
output_dir               : /content
pattern                  : *.csv
compute_optimal          : True
timezone                 : America/Monterrey
log_level                : INFO
random_seed              : None
max_workers              : None
python_version           : 3.12.12
numpy_version            : 2.0.2
pandas_version           : 2.2.2
timestamp                : 2025-12-22T17:28:58.916033-06:00


### 12.2 Mount Google Drive (Colab only)

In [37]:
if IN_COLAB:
    drive.mount('/content/drive')
    print("Google Drive mounted")
else:
    print("Not running in Colab, skipping Drive mount")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted


### 12.3 Initialize Components

In [38]:
csv_dir = config.data_dir
CAPACITY = config.capacity

registry = HeuristicRegistry()
optimal_solver = OptimalSolver()
executor = ExperimentExecutor(registry, optimal_solver)

print(f"\n Initialized with {len(registry.get_names())} heuristics:")
for i, name in enumerate(registry.get_names(), 1):
    print(f"  {i}. {name}")


 Initialized with 8 heuristics:
  1. default
  2. max_profit
  3. max_profit_per_weight
  4. min_weight
  5. max_profit_per_weight_tiebreak_profit
  6. max_profit_per_weight_tiebreak_weight
  7. max_profit_tiebreak_weight
  8. min_weight_tiebreak_profit


### 12.4 Process All Instances

In [39]:
logger.info(f"Processing instances from: {csv_dir}")
logger.info(f"Parallel processing: {config.enable_parallel}, Max workers: {config.max_workers or 'auto'}")

results = executor.execute_directory(
    directory=csv_dir,
    capacity=CAPACITY,
    pattern=config.pattern,
    compute_optimal=config.compute_optimal,
    parallel=config.enable_parallel,
    max_workers=config.max_workers
)

logger.info(f"Processed {len(results)} instances")

INFO:__main__:Processing instances from: /content/drive/MyDrive/PlataZarateBayliss-Instances_extracted_csv
INFO:__main__:Parallel processing: True, Max workers: auto
INFO:__main__.ExperimentExecutor:Found 500 instances in /content/drive/MyDrive/PlataZarateBayliss-Instances_extracted_csv
INFO:__main__.ExperimentExecutor:Starting parallel execution with 6 workers


Processing 500 instances in parallel (6 workers)...


INFO:__main__.ExperimentExecutor:Completed 10/500


Completed 10/500


INFO:__main__.ExperimentExecutor:Completed 20/500
INFO:__main__.ExperimentExecutor:Completed 30/500
INFO:__main__.ExperimentExecutor:Completed 40/500
INFO:__main__.ExperimentExecutor:Completed 50/500


...

Completed 380/500
Completed 390/500
Completed 400/500
Completed 410/500
Completed 420/500


INFO:__main__.ExperimentExecutor:Completed 430/500
INFO:__main__.ExperimentExecutor:Completed 440/500
INFO:__main__.ExperimentExecutor:Completed 450/500
INFO:__main__.ExperimentExecutor:Completed 460/500
INFO:__main__.ExperimentExecutor:Completed 470/500


Completed 430/500
Completed 440/500
Completed 450/500
Completed 460/500
Completed 470/500


INFO:__main__.ExperimentExecutor:Completed 480/500
INFO:__main__.ExperimentExecutor:Completed 490/500
INFO:__main__.ExperimentExecutor:Completed 500/500
INFO:__main__.ExperimentExecutor:Parallel execution completed: 500/500 instances
INFO:__main__:Processed 500 instances


Completed 480/500
Completed 490/500
Completed 500/500


### 12.5 Convert Results to DataFrame

In [40]:
converter = ResultsConverter()
df = converter.to_dataframe(results)
execution_times = converter.extract_execution_times(results)

print("\nFirst few rows:")
print(df.head())
print(f"\nDataFrame shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")


First few rows:
                   instance  capacity  default  max_profit  \
0  TEST_GA-DEF_EASY_100_000        64      686         612   
1  TEST_GA-DEF_EASY_100_001        64      704         311   
2  TEST_GA-DEF_EASY_100_002        64      571         272   
3  TEST_GA-DEF_EASY_100_003        64     1046         711   
4  TEST_GA-DEF_EASY_100_004        64      691         352   

   max_profit_per_weight  min_weight  max_profit_per_weight_tiebreak_profit  \
0                    613         613                                    613   
1                    643         507                                    643   
2                    513         486                                    513   
3                    975         975                                    975   
4                    625         579                                    625   

   max_profit_per_weight_tiebreak_weight  max_profit_tiebreak_weight  \
0                                    613                       

### 12.6 Save Results

In [41]:
output_file = save_results(
    df=df,
    output_path=Path(config.output_dir) / 'knapsack_results.csv',
    include_timestamp=True,
    base_name=f'heuristic_results_cap{CAPACITY}'
)

print(f"\nResults saved to: {output_file}")

INFO:__main__:Saved results: /content/knapsack_results_20251222_232907.csv



Results saved to: /content/knapsack_results_20251222_232907.csv


### 12.7 Generate Report

In [42]:
timestamp = datetime.now(pytz.timezone(config.timezone)).strftime('%Y%m%d_%H%M%S')
report_path = Path(config.output_dir) / f'experiment_report_CAP{CAPACITY}_{timestamp}.txt'
config_path = Path(config.output_dir) / f'experiment_config_{timestamp}.json'

config.save_to_file(str(config_path))

formatter = ReportFormatter()
analyzer = StatisticalAnalyzer()
report_gen = ReportGenerator(formatter, analyzer)

report_gen.generate(
    df=df,
    execution_times=execution_times,
    config=config.to_dict(),
    output_path=str(report_path)
)

print(f"\nReport saved to: {report_path}")
print(f"Configuration saved to: {config_path}")

INFO:__main__:Configuration saved to /content/experiment_config_20251222_172907.json
INFO:__main__.ReportGenerator:Report generated: /content/experiment_report_CAP64_20251222_172907.txt



Report saved to: /content/experiment_report_CAP64_20251222_172907.txt
Configuration saved to: /content/experiment_config_20251222_172907.json


### 12.8 Display Summary Statistics

In [43]:
print("\n" + "="*80)
print("EXPERIMENT SUMMARY")
print("="*80)

heuristic_columns = [col for col in df.columns if col not in ['instance', 'capacity', 'optimal']]
stats_df = analyzer.compute_statistics(df, heuristic_columns)

print("\nHeuristic Performance Statistics:")
print(stats_df.to_string(index=False))

if 'optimal' in df.columns:
    gap_stats = analyzer.compute_gap_statistics(df, heuristic_columns)
    print("\nOptimality Gap Analysis:")
    print(gap_stats[['heuristic', 'mean_gap', 'optimal_found']].to_string(index=False))

dominant = analyzer.identify_dominant_heuristic(df, heuristic_columns)
print("\nDominant Heuristic (times achieved best solution):")
print(dominant.to_string())


EXPERIMENT SUMMARY

Heuristic Performance Statistics:
                            heuristic    mean        std  min  max  median
                              default 378.130 239.194191    4 1074   297.5
                           max_profit 524.438 241.761203  127 1303   492.5
                max_profit_per_weight 892.068 238.096407  394 1476   894.0
                           min_weight 696.612 284.167528  184 1498   625.5
max_profit_per_weight_tiebreak_profit 892.142 237.544490  394 1476   894.0
max_profit_per_weight_tiebreak_weight 892.172 238.451296  394 1476   894.0
           max_profit_tiebreak_weight 571.094 239.951175  128 1303   543.0
           min_weight_tiebreak_profit 724.798 273.410651  184 1511   655.5

Optimality Gap Analysis:
                            heuristic  mean_gap  optimal_found
                              default 55.508541            101
                           max_profit 41.957777            104
                max_profit_per_weight  6.641649        